In [1]:
import cv2
import mediapipe as mp

In [2]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np


def draw_landmarks_on_image(rgb_image, detection_result):
  pose_landmarks_list = detection_result.pose_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      pose_landmarks_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style())
  return annotated_image

In [3]:
# STEP 1: Import the necessary modules.
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# STEP 2: Create an PoseLandmarker object.
base_options = python.BaseOptions(model_asset_path='models/mediapipe/pose_landmarker_lite.task')
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    output_segmentation_masks=True)
detector = vision.PoseLandmarker.create_from_options(options)


In [4]:
import cv2
import mediapipe as mp

def cv2_to_mediapipe(cv2_image):
  """Converts a CV2 image to a MediaPipe Image.

  Args:
    cv2_image: The input CV2 image.

  Returns:
    The converted MediaPipe Image.
  """

  # Convert BGR to RGB
  rgb_image = cv2.cvtColor(cv2_image, cv2.COLOR_BGR2RGB)
  # Create a MediaPipe Image
  mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_image)
  return mp_image

In [5]:
def resize_video(video_path, scale_factor):
  """Resizes a video by a given scale factor.

  Args:
    video_path: Path to the input video file.
    scale_factor: Scaling factor (e.g., 0.5 for half size).

  Returns:
    A generator yielding resized frames.
  """

  cap = cv2.VideoCapture(video_path)

  while cap.isOpened():
    success, frame = cap.read()
    if not success:
      break

    width = int(frame.shape[1] * scale_factor)
    height = int(frame.shape[0] * scale_factor)
    resized_frame = cv2.resize(frame, (width, height))

    yield resized_frame

  cap.release()

In [6]:
# Example usage:
video_path = 'videos/squats.mp4'
scale_factor = 0.25

for frame in resize_video(video_path, scale_factor):
  # Process the resized frame with MediaPipe
  
  # STEP 3: Load the input image.
  image = cv2_to_mediapipe(frame)

  # STEP 4: Detect pose landmarks from the input image.
  detection_result = detector.detect(image)

  # STEP 5: Process the detection result. In this case, visualize it.
  annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
  # Convert back to BGR for OpenCV visualization
  cv2.imshow("output", cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))

  if cv2.waitKey(5) & 0xFF == 27:
    break

cv2.destroyAllWindows()

c:\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
